In [ ]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error
)
from xgboost import XGBRegressor

pd.options.display.max_columns = None

from scripts.data_processing import (
    load_uci, load_tidepool_dummy, 
    load_so_pump, 
    load_so_cgm
)

In [ ]:
def split_train_validate(df, minutes=30, test_fraction=0.2):
    test_size = int(df.shape[0] * test_fraction)
    df_train = df.iloc[0:-test_size]
    df_val   = df.iloc[-test_size:]
    print(f'train size: {len(df_train)}')
    print(f'test size: {len(df_val)}')
    
    target_name = str(minutes) + '_minutes'
    X_train  = df_train.drop(columns=[target_name])
    y_train  = df_train[target_name]
    
    X_val    = df_val.drop(columns=[target_name])
    y_val    = df_val[target_name]
    
    return X_train, X_val, y_train, y_val


def type_check(df):
    for col in ['timestamp', 'value', 'below_threshold']:
        assert pd.api.types.is_numeric_dtype(df[col])
    
    return None


def feature_engineer(df, minutes=30, n_historical_cols=2):
    """Adds most recent CGM rates as columns"""
    type_check(df)
    
    for x in range(1, n_historical_cols+1):
        df[['prev_val', 'prev_time']] = df[['value', 'timestamp']].shift(x)
        df[f'prev_trend_{x}'] = (
            df['prev_val'].divide(df['timestamp'] - df['prev_time']))
        df = df.drop(columns=['prev_val', 'prev_time'])

    # remove nans
    og_len = len(df)
    df = df.loc[~df[f'prev_trend_{n_historical_cols}'].isna()]
    n_dropped = og_len - len(df)
    assert n_dropped <= n_historical_cols

    return df


def append_future_target_col(df, minutes):
    """Append target column for machine learning model"""
    seconds = minutes * 60
    
    df[f'{minutes}_minutes'] = np.interp(
        df['timestamp'].add(seconds), df['timestamp'],
        df['value']
    )
    
    max_valid_time = df['timestamp'].max() - seconds
    df.loc[df['timestamp'] > max_valid_time, f'{minutes}_minutes'] = np.nan
    
    # remove nans
    og_len = len(df)
    df = df.loc[~df[f'{minutes}_minutes'].isna()]
    n_dropped = og_len - len(df)
    assert n_dropped < 10
    
    return df


def build_model(df, minutes=30):
    df = feature_engineer(df.copy(), minutes)
    df = append_future_target_col(df.copy(), minutes)
    
    X_train, X_val, y_train, y_val = \
        split_train_validate(df.copy(), minutes=minutes)
    
    param_grid = {  
        'learning_rate': [0.05, 0.06, 0.07, 0.08],
        'n_estimators':  [40, 50, 60],
        'max_depth': [2],
    }

    gridsearch = GridSearchCV(XGBRegressor(),
                              param_grid=param_grid, 
                              # scoring='roc_auc', 
                              cv=3, n_jobs=-1,
                              return_train_score=True, verbose=10)
    gridsearch.fit(X_train, y_train)
    print('best estimator:', gridsearch.best_estimator_)
    print(gridsearch.cv_results_['mean_train_score'].mean(),
          gridsearch.cv_results_['mean_test_score'].mean())
    y_pred = gridsearch.predict(X_val)
    print('mae:', mean_absolute_error(y_val, y_pred))
    print('rmse:', np.sqrt(mean_squared_error(y_val, y_pred)))
    
    return gridsearch

def baseline_rmse(df, minutes=30):
    """Baseline RMSE assuming the target value will be the same
    as the current value
    """
    df = load_so_cgm()
    df = feature_engineer(df.copy(), minutes)
    df = append_future_target_col(df.copy(), minutes)
    
    mse = sum((df[f'{minutes}_minutes'] - 
               df['value']).pow(2)) / len(df)
    
    return np.sqrt(mse)

### Compute baseline MSE

In [ ]:
for minute in range(5, 31, 5):
    print(baseline_rmse(df=load_so_cgm(), minutes=minute))

In [ ]:
# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bytree=1, gamma=0, importance_type='gain',
#        learning_rate=0.06, max_delta_step=0, max_depth=2,
#        min_child_weight=1, missing=None, n_estimators=60, n_jobs=1,
#        nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
#        reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
#        subsample=1)
# gave us +/- 18.66 mg/dL

## Write Models

In [ ]:
df = load_so_cgm()
models = []
for minutes in range(5, 31, 5):
    with open('diabetesmanager/ml_models/model_' + str(minutes) + '_minutes.pkl', 'wb') as f:
        model = build_model(df.copy(), minutes)
        pickle.dump(model, f)
        models.append(model)

## Read Models

In [ ]:
from pathlib import Path

MODEL_DIR = Path('diabetesmanager/ml_models')
models = []
for model_path in MODEL_DIR.iterdir():
    if str(model_path).endswith('.pkl'):
        with open(model_path, 'rb') as f:
            models.append(pickle.load(f))

## Plotting

In [ ]:
import matplotlib.pyplot as plt

df = load_so_cgm()
df = preprocess(df.copy(), minutes=5)
df = feature_engineer(df.copy(), minutes)
df = df[[col for col in df if '_minutes' not in col]]

last_seconds = 30 * 60
seconds_ago = df['timestamp'].max() - last_seconds
for i, model in enumerate(models):
    print((i+1)*5, 'minutes')
    y_pred = model.predict(df.loc[df['timestamp'] >= seconds_ago])
    
    # actual
    plt.plot(df.loc[df['timestamp'] >= seconds_ago, 'timestamp'], 
             df.loc[df['timestamp'] >= seconds_ago, 'value'], color='b', marker='P')
    
    # prediction
    plt.plot(df.loc[df['timestamp'] >= seconds_ago,  'timestamp'] + (i+1)*5*60, 
             y_pred, color='g', marker='d')
    plt.show()